In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_style("darkgrid")

In [86]:
data = pd.read_csv('ballons.csv')

def prior_probability(df):
      
    rows = len(df)
    values = df['Inflated'].value_counts()
    prior = []

    for i in values:
        prob = i/rows
        prior.append(prob)
        
    return prior     

def naive_bayes_classifer(df, X_test, Y_test): 
    
    feature_prob_given_true = []
    feature_prob_given_false = []
    
    features = df.columns[:-1]
    rows = len(df[Y_test])
    class_value = df[Y_test].value_counts()

    for feature in features:
        distinct = df[feature].unique()     # ['YELLOW', 'RED', 'GREEN']
        
        for f_value in distinct:            # 'YELLOW'
            feature_value_true = 0
            feature_value_false = 0
            
            for i in range(len(df)):        # for each row
                if df[Y_test][i] == 'T' and df[feature][i] == f_value:    # if target is true
                    feature_value_true+=1
                elif df[Y_test][i] == 'F' and df[feature][i] == f_value:    # if target is false
                    feature_value_false+=1
                                
            feature_prob_given_true.append({f_value: feature_value_true/class_value['T']})
            feature_prob_given_false.append({f_value: feature_value_false/class_value['F']})
    
    
    porsterior_T = 1
    porsterior_F = 1
    for i in range(len(X_test)):
        x = X_test[i]
        
        mle = 1
        for f_val in feature_prob_given_true:   # f_val = {'YELLOW': 0.5}
            for key, value in f_val.items():    # key = 'YELLOW', value = 0.5
                if x == key:                    # if x = 'YELLOW'
                    mle *= value                # mle = 0.5
                     
        porsterior_T *= mle                     # P(YELLOW|T) = 0.5, P(LARGE|T) = 0.5, P(STRETCH|T) = 0.5, P(ADULT|T) = 0.5
        
        mle = 1
        for f_val in feature_prob_given_false:  # f_val = {'YELLOW': 0.5}
            for key, value in f_val.items():    # key = 'YELLOW', value = 0.5
                if x == key:                    # if x = 'YELLOW'
                    mle *= value                # mle = 0.5
                    
                
        porsterior_F *= mle                     # P(YELLOW|F) = 0.5, P(LARGE|F) = 0.5, P(STRETCH|F) = 0.5, P(ADULT|F) = 0.5
        
    porsterior_T *= prior_probability(data)[0]  # P(T|YELLOW, LARGE, STRETCH, ADULT) = P(YELLOW|T) * P(LARGE|T) * P(STRETCH|T) * P(ADULT|T) * P(T)
    porsterior_F *= prior_probability(data)[1]  # P(F|YELLOW, LARGE, STRETCH, ADULT) = P(YELLOW|F) * P(LARGE|F) * P(STRETCH|F) * P(ADULT|F) * P(F)
    
    return (porsterior_T > porsterior_F)        # return True if P(T|YELLOW, LARGE, STRETCH, ADULT) > P(F|YELLOW, LARGE, STRETCH, ADULT)


p = prior_probability(data)
m = naive_bayes_classifer(df=data, X_test=['YELLOW', 'LARGE', 'STRETCH', 'CHILD'], Y_test='Inflated')
print(m)


False
